## Build a Retrieval Augmented Generation (RAG) App: Part 1
As per this [tutorial](https://python.langchain.com/docs/tutorials/rag/)

One of the most powerful applications enabled by LLMs is sophisticated question-answering (Q&A) chatbots. These are applications that can answer questions about specific source information. These applications use a technique known as Retrieval Augmented Generation, or [RAG](https://python.langchain.com/docs/concepts/rag/).

This is a multi-part tutorial:

Part 1 (this guide) introduces RAG and walks through a minimal implementation.
[Part 2](https://python.langchain.com/docs/tutorials/qa_chat_history/) extends the implementation to accommodate conversation-style interactions and multi-step retrieval processes.
This tutorial will show how to build a simple Q&A application over a text data source. Along the way we’ll go over a typical Q&A architecture and highlight additional resources for more advanced Q&A techniques. We’ll also see how LangSmith can help us trace and understand our application. LangSmith will become increasingly helpful as our application grows in complexity.

If you're already familiar with basic retrieval, you might also be interested in this [high-level overview](https://python.langchain.com/docs/concepts/retrieval/) of different retrieval techniques.

Note: Here we focus on Q&A for unstructured data. If you are interested for RAG over structured data, check out our tutorial on doing [question/answering over SQL data](https://python.langchain.com/docs/tutorials/sql_qa/).

## Overview
A typical RAG application has two main components:

Indexing: a pipeline for ingesting data from a source and indexing it. This usually happens offline.

Retrieval and generation: the actual RAG chain, which takes the user query at run time and retrieves the relevant data from the index, then passes that to the model.

Note: the indexing portion of this tutorial will largely follow the [semantic search tutorial](https://python.langchain.com/docs/tutorials/retrievers/).

The most common full sequence from raw data to answer looks like:

#### Indexing
- Load: First we need to load our data. This is done with [Document Loaders](https://python.langchain.com/docs/concepts/document_loaders/).
- Split: [Text splitters](https://python.langchain.com/docs/concepts/text_splitters/) break large Documents into smaller chunks. This is useful both for indexing data and passing it into a model, as large chunks are harder to search over and won't fit in a model's finite context window.
- Store: We need somewhere to store and index our splits, so that they can be searched over later. This is often done using a VectorStore and Embeddings model.

#### Retrieval and Generation
- Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/docs/concepts/retrievers/).
- Generate: A ChatModel / [LLM](https://python.langchain.com/docs/concepts/text_llms/) produces an answer using a prompt that includes both the question with the retrieved data

Once we've indexed our data, we will use LangGraph as our orchestration framework to implement the retrieval and generation steps.

In [2]:
import os
from dotenv import load_dotenv

# Loading the .env file. Using full path to use it in the interactive shell
load_dotenv("C:\\Users\\MarcusChiri\\Dropbox\\Repositories\\Langchain\\.env")

# Loarding the environment variables
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"

### Components
We will need to select three components from LangChain's suite of integrations.

Select chat model:

In [1]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

Select [embeddings model](https://python.langchain.com/docs/integrations/text_embedding/):

In [1]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

Select [vector store](https://python.langchain.com/docs/integrations/vectorstores/):

In [ ]:
from langchain_postgres import PGVector
import os

vector_store = PGVector(
    embeddings=embeddings,
    collection_name="testing_rag",
    # when we use the PGVector class, it creates the collection if inexistent
    # in PostgreSQL, the collection names are stored in a table called langchain_pg_collection
    # the actual embedding vectors are stored in a table called langchain_pg_embedding, which also contains the collection name, document text and the metadata
    connection=os.getenv("PSQL_DB_URL")
)

### Preview
In this guide we’ll build an app that answers questions about the website's content. The specific website we will use is the [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) blog post by Lilian Weng, which allows us to ask questions about the contents of the post.

We can create a simple indexing pipeline and RAG chain to do this in ~50 lines of code.

# If you're running the whole tutorial, don't run this cell below!
This cell is the complete build, just for overview. We'll build and explain each component below.

In [ ]:
# import bs4
# from langchain import hub
# from langchain_community.document_loaders import WebBaseLoader
# from langchain_core.documents import Document
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langgraph.graph import START, StateGraph
# from typing_extensions import List, TypedDict

# # Load and chunk contents of the blog
# loader = WebBaseLoader(
#     web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(
#             class_=("post-content", "post-title", "post-header")
#         )
#     ),
# )
# docs = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=1000, chunk_overlap=200)
# all_splits = text_splitter.split_documents(docs)

# # Index chunks
# _ = vector_store.add_documents(documents=all_splits)

# # Define prompt for question-answering
# prompt = hub.pull("rlm/rag-prompt")


# # Define state for application
# class State(TypedDict):
#     question: str
#     context: List[Document]
#     answer: str


# # Define application steps
# def retrieve(state: State):
#     retrieved_docs = vector_store.similarity_search(state["question"])
#     return {"context": retrieved_docs}


# def generate(state: State):
#     docs_content = "\n\n".join(doc.page_content for doc in state["context"])
#     messages = prompt.invoke(
#         {"question": state["question"], "context": docs_content})
#     response = llm.invoke(messages)
#     return {"answer": response.content}


# # Compile application and test
# graph_builder = StateGraph(State).add_sequence([retrieve, generate])
# graph_builder.add_edge(START, "retrieve")
# graph = graph_builder.compile()

In [ ]:
# Don't run the cell below if you're going through the whole tutorial, which is detailed below.

In [ ]:
# response = graph.invoke({"question": "What is Task Decomposition?"})
# print(response["answer"])

## Detailed walkthrough of the code above

### 1) Indexing
#### 1.1 Loading documents
We need to first load the blog post contents. We can use [DocumentLoaders](https://python.langchain.com/docs/concepts/document_loaders/) for this, which are objects that load in data from a source and return a list of [Document](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.base.Document.html) objects.

In this case we’ll use the [WebBaseLoader](https://python.langchain.com/docs/integrations/document_loaders/web_base/), which uses urllib to load HTML from web URLs and BeautifulSoup to parse it to text. We can customize the HTML -> text parsing by passing in parameters into the BeautifulSoup parser via bs_kwargs (see [BeautifulSoup docs](https://beautiful-soup-4.readthedocs.io/en/latest/#beautifulsoup)). In this case only HTML tags with class “post-content”, “post-title”, or “post-header” are relevant, so we’ll remove all others.

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# Only keep post title, headers, and content from the full HTML.
bs4_strainer = bs4.SoupStrainer(
    class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

assert len(docs) == 1
print(f"Total characters: {len(docs[0].page_content)}")

USER_AGENT environment variable not set, consider setting it to identify your requests.


Total characters: 43130


In [ ]:
print(*docs[0], sep="\n\n")

('id', None)

('metadata', {'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'})

('page_content', '\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn

Go deeper
DocumentLoader: Object that loads data from a source as list of Documents.

- [Docs](https://python.langchain.com/docs/how_to/#document-loaders): Detailed documentation on how to use DocumentLoaders.
- [Integrations](https://python.langchain.com/docs/integrations/document_loaders/): 160+ integrations to choose from.
- [Interface](https://python.langchain.com/api_reference/core/document_loaders/langchain_core.document_loaders.base.BaseLoader.html): API reference for the base interface.

### 1.2 Splitting documents
Our loaded document is over 42k characters which is too long to fit into the context window of many models. Even for those models that could fit the full post in their context window, models can struggle to find information in very long inputs.

To handle this we’ll split the Document into chunks for embedding and vector storage. This should help us retrieve only the most relevant parts of the blog post at run time.

As in the [semantic search tutorial](https://python.langchain.com/docs/tutorials/retrievers/), we use a [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/how_to/recursive_text_splitter/), which will recursively split the document using common separators like new lines until each chunk is the appropriate size. This is the recommended text splitter for generic text use cases.

In [17]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 66 sub-documents.


Go deeper

TextSplitter: Object that splits a list of Documents into smaller chunks. Subclass of DocumentTransformers.

Learn more about splitting text using different methods by reading the [how-to docs](https://python.langchain.com/docs/how_to/#text-splitters):
- [Code (py or js)](https://python.langchain.com/docs/integrations/document_loaders/source_code/)
- [Scientific papers](https://python.langchain.com/docs/integrations/document_loaders/grobid/)
- [Interface](https://python.langchain.com/api_reference/text_splitters/base/langchain_text_splitters.base.TextSplitter.html): API reference for the base interface.

DocumentTransformer: Object that performs a transformation on a list of Document objects.

- [Docs](https://python.langchain.com/docs/how_to/#text-splitters): Detailed documentation on how to use DocumentTransformers
- [Integrations](https://python.langchain.com/docs/integrations/document_transformers/)
- [Interface](https://python.langchain.com/api_reference/core/documents/langchain_core.documents.transformers.BaseDocumentTransformer.html): API reference for the base interface.

### 1.3 Storing documents
Now we need to index our 66 text chunks so that we can search over them at runtime. Following the semantic search tutorial, our approach is to [embed](https://python.langchain.com/docs/concepts/embedding_models/) the contents of each document split and insert these embeddings into a [vector store](https://python.langchain.com/docs/concepts/vectorstores/). Given an input query, we can then use vector search to retrieve relevant documents.

We can embed and store all of our document splits in a single command using the vector store and embeddings model selected at the [start of the tutorial](https://python.langchain.com/docs/tutorials/rag/#components).

In [ ]:
# not necessary if docs are already in the database
document_ids = vector_store.add_documents(documents=all_splits)



print(document_ids[:3])

['bb7ef1eb-1fbe-49f4-ba14-f4d5c2f98e58', 'f08d9033-9b36-4bb0-afe4-4be402e9e73e', '354177c2-2834-4370-b255-c2ef090bd29f']


Go deeper

Embeddings: Wrapper around a text embedding model, used for converting text to embeddings.

- [Docs](https://python.langchain.com/docs/how_to/embed_text/): Detailed documentation on how to use embeddings.
- [Integrations](https://python.langchain.com/docs/integrations/text_embedding/): 30+ integrations to choose from.
- [Interface](https://python.langchain.com/api_reference/core/embeddings/langchain_core.embeddings.Embeddings.html): API reference for the base interface.

VectorStore: Wrapper around a vector database, used for storing and querying embeddings.

- [Docs](https://python.langchain.com/docs/how_to/vectorstores/): Detailed documentation on how to use vector stores.
- [Integrations](https://python.langchain.com/docs/integrations/vectorstores/): 40+ integrations to choose from.
- [Interface](https://python.langchain.com/api_reference/core/vectorstores/langchain_core.vectorstores.base.VectorStore.html): API reference for the base interface.

This completes the Indexing portion of the pipeline. At this point we have a query-able vector store containing the chunked contents of our blog post. Given a user question, we should ideally be able to return the snippets of the blog post that answer the question.

## 2. Retrieval and Generation
Now let’s write the actual application logic. We want to create a simple application that takes a user question, searches for documents relevant to that question, passes the retrieved documents and initial question to a model, and returns an answer.

For generation, we will use the chat model selected at the start of the tutorial.

We’ll use a prompt for RAG that is checked into the LangChain prompt hub ([here](https://smith.langchain.com/hub/rlm/rag-prompt?_gl=1*15bkrfm*_ga*MTE5ODA2NjM0Ni4xNzQ0OTgyMzI2*_ga_47WX3HKKY2*MTc0NTc0ODY1Mi4xMC4xLjE3NDU3NDg3MjkuMC4wLjA.)).

**NB**: The Tutorial doesn't explicit that, but the prompt we're getting from the hub is the following:

"
You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:
"

In [ ]:
from langchain import hub

# prompt = hub.pull("rlm/rag-prompt")

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:",
        )
    ]
)


# going forward with the tutorial as is
example_messages = prompt.invoke(
    {"context": "(context goes here)", "question": "(question goes here)"}
).to_messages()

assert len(example_messages) == 1
print(example_messages[0].content)

You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: (question goes here)
Context: (context goes here)
Answer:
[SystemMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: (question goes here)\nContext: (context goes here)\nAnswer:", additional_kwargs={}, response_metadata={})]
